In [1]:
import pandas as pd
import numpy as np
import matplotlib


In [255]:
df = pd.read_csv('./files/21-05-06_160653_calib_SN04_DACSet000_20x.csv')

#remove some columns not exploitable
df.drop(['Bus','No', 'ID (hex)', 'Message', 'ASCII'], inplace=True, axis=1)
df[0:10]


,Time (abs),State,Length,Data (hex)
0,4346.454.319.0,S,2,01 01
1,4346.455.060.0,,7,01 00 00 00 00 00 00
2,4346.455.690.0,,7,21 00 00 00 00 00 00
3,4347.246.327.0,S,2,01 02
4,4347.247.073.0,,7,01 80 00 80 00 80 00
5,4347.247.708.0,,7,21 80 00 00 00 00 00
6,4349.246.321.0,S,2,01 06
7,4349.247.072.0,,7,01 00 00 00 00 00 00
8,4349.247.702.0,,7,21 00 00 00 00 00 00
9,4349.248.333.0,,7,61 00 00 00 00 00 00


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Time (abs)  285 non-null    object
 1   State       285 non-null    object
 2   Length      285 non-null    int64 
 3   Data (hex)  285 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.0+ KB


In [80]:
df['tmp'] = df['State'].apply(lambda x: 'S' in x )
mssgidx = df[df['tmp'] == True].index.tolist()
df.drop(['tmp'], inplace=True, axis=1)
mssgidx

[0,
 3,
 6,
 11,
 16,
 19,
 21,
 23,
 25,
 30,
 33,
 38,
 43,
 46,
 51,
 56,
 59,
 64,
 69,
 72,
 77,
 82,
 85,
 90,
 95,
 98,
 103,
 108,
 111,
 116,
 121,
 124,
 129,
 134,
 137,
 142,
 147,
 150,
 155,
 160,
 163,
 168,
 173,
 176,
 181,
 186,
 189,
 194,
 199,
 202,
 207,
 212,
 215,
 220,
 225,
 228,
 233,
 238,
 241,
 246,
 251,
 254,
 259,
 264,
 267,
 272,
 277,
 280]

In [81]:
df

,Time (abs),State,Length,Data (hex)
0,4346.454.319.0,S,2,01 01
1,4346.455.060.0,,7,01 00 00 00 00 00 00
2,4346.455.690.0,,7,21 00 00 00 00 00 00
3,4347.246.327.0,S,2,01 02
4,4347.247.073.0,,7,01 80 00 80 00 80 00
...,...,...,...,...
280,4398.097.417.0,S,2,01 04
281,4398.098.169.0,,7,01 00 00 00 00 00 00
282,4398.098.799.0,,7,21 00 00 00 00 00 00
283,4398.099.429.0,,7,61 00 00 00 00 00 00


In [140]:
df.to_csv('calib_v6_SN04_fromscript_DACset000.txt', sep=',', index=False, header=False, quotechar = ' ')


In [242]:


file1 = open('calib_v6_SN04_fromscript_DACset000.txt', 'r')
#file1 = open('./files/21-05-06_160653_calib_SN04_DACSet000_20x.csv', 'r')
Lines = file1.readlines()

#definition of the dictionary with the possible requests which can be found and the expected length of the answers
d_request = {'0101' : 2, '0102': 2, '0106': 4, '0105': 4, '0109': 2, '0100': 2, '0104': 4 }

#define a global list of dictionary
d_list = []

# loop over lines. The iterator i is needed to be able to extract the lines containing the answers
for i in range(0, len(Lines)):
    line = Lines[i]
    #print(line)
    singles = line.split(',') 
    singles[1] = singles[1].strip()
    #print(singles[1])    
    #print(singles)    

    #build a tmp dictionary which will be then saved in the global list of dictionary d_list
    tmp_dic = {'timestamp':singles[0], 'state': singles[1], 'mssg_length':singles[2], 'mssg':singles[3] } 
    


    list(tmp_dic)
    if (tmp_dic.get('state')== 'S'):
        message = tmp_dic.get('mssg').strip()
        message = message.replace(' ', '')
        #replace mssg in the dictionary
        tmp_dic.update(mssg=message)

        #print('this is a request:', tmp_dic.get('state'))
        #print('this is the message:', message)

        for value in d_request.keys():
            #print(value)
            if (message== value):
                #print(line)
                #print('found request: ', value)
                #print('the answer length is: ', d_request.get(value))
                answer_length = d_request.get(value)
                #print('there are the next ',  answer_length,' lines :')
                
        
                #prepare a list to record the answer
                aws =[] 

                #loop over the line of the answer: 
                for a_i in range(1, answer_length+1):
                    #print(' answer:', Lines[i+a_i])
                    awsline = Lines[i+a_i].split(',') 
                    aws.append(awsline[3].strip())    
                    
                #print('printing the answer: ', aws)
                tmp_dic['answer'] = aws
                #print('***** ', tmp_dic)
                d_list.append(tmp_dic)
                #print('print dictionary list', d_list)


print(d_list[0])

{'timestamp': '4346.454.319.0', 'state': 'S', 'mssg_length': '2', 'mssg': '0101', 'answer': ['01  00  00  00  00  00  00', '21  00  00  00  00  00  00']}


In [243]:
d_list[0:10]

[{'timestamp': '4346.454.319.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0101',
  'answer': ['01  00  00  00  00  00  00', '21  00  00  00  00  00  00']},
 {'timestamp': '4347.246.327.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0102',
  'answer': ['01  80  00  80  00  80  00', '21  80  00  00  00  00  00']},
 {'timestamp': '4349.246.321.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0106',
  'answer': ['01  00  00  00  00  00  00',
   '21  00  00  00  00  00  00',
   '61  00  00  00  00  00  00',
   '41  00  00  00  00  00  00']},
 {'timestamp': '4352.358.266.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0109',
  'answer': ['01  00  00  A5  F6  00  00', '21  00  00  08  00  E6  CB']},
 {'timestamp': '4378.663.928.0',
  'state': 'S',
  'mssg_length': '2',
  'mssg': '0105',
  'answer': ['01  E6  AA  E6  BB  E6  B4',
   '21  E6  AE  E6  C2  E6  B9',
   '41  E6  AF  E6  B7  E7  03',
   '61  E6  CE  E6  D8  E6  DE']},
 {'timestamp': '4378.864.565.0',
  'state'

In [245]:
#print(d_list[]['mssg'])
#rint(d_list[]['mssg'])
next(item for item in d_list if item[['mssg'] == '0105')

{'timestamp': '4378.663.928.0',
 'state': 'S',
 'mssg_length': '2',
 'mssg': '0105',
 'answer': ['01  E6  AA  E6  BB  E6  B4',
  '21  E6  AE  E6  C2  E6  B9',
  '41  E6  AF  E6  B7  E7  03',
  '61  E6  CE  E6  D8  E6  DE']}

In [340]:
#define a function which take in input a list of dictionary and 
# search for the request 0105 and separate the channels
# --> TO DO: for the request 0105 return a list of list with the ADC values per channel

def getADCperchannel(mylist):

    mylist = mylist.copy()

    #create the dictionary which will collect all the channels infos
    #d_0105 = {}
    l_ch0, l_ch1, l_ch2 = [], [], []
    l_ch3, l_ch4, l_ch5 = [], [], []
    l_ch6, l_ch7, l_ch8 = [], [], []
    l_ch9, l_ch10, l_ch11 = [], [], []

    # define the list of list which will contain all the list of single channels values recording their ADC values
    l_allch = []

    for d in mylist:
        if (d.get('mssg')=='0105'):
            answ = d.get('answer')

            #check if the lenght is ok (it should be a list of 4 elements)
            if (len(answ) !=4): abort

            for answ_items in answ:
                answ_items = answ_items.split(' ')
                #print(answ_items)
            #answ_singles = answ[0].split(' ')
            #print(answ_singles)
         
            #if (answ_singles[0] == '01'):
            #    ch_dic = {'channels': answ_singles[0], 
            #    'ch0':answ_singles[2]+answ_singles[4] , 
            #    'ch1':answ_singles[6]+answ_singles[8] , 
            #    'ch2':answ_singles[10]+answ_singles[12]}
            #    d_0105.append(ch_dic)

            #elif (answ_singles[0] == '21'):
            #    ch_dic = {'channels': answ_singles[0], 
            #    'ch3':answ_singles[2]+answ_singles[4] , 
            #    'ch4':answ_singles[6]+answ_singles[8] , 
            #    'ch5':answ_singles[10]+answ_singles[12]}
            #    d_0105.append(ch_dic)

            #elif (answ_singles[0] == '41'):
            #    ch_dic = {'channels': answ_singles[0], 
            #    'ch6':answ_singles[2]+answ_singles[4] , 
            #    'ch7':answ_singles[6]+answ_singles[8] , 
            #    'ch8':answ_singles[10]+answ_singles[12]}
            #    d_0105.append(ch_dic)

            #elif (answ_singles[0] == '61'):
            #    ch_dic = {'channels': answ_singles[0], 
            #    'ch9':answ_singles[2]+answ_singles[4] , 
            #    'ch10':answ_singles[6]+answ_singles[8] , 
            #    'ch11':answ_singles[10]+answ_singles[12]}
            #   d_0105.append(ch_dic)

                if (answ_items[0] == '01'):
                    l_ch0.append(answ_items[2]+answ_items[4])
                    l_ch1.append(answ_items[6]+answ_items[8])
                    l_ch2.append(answ_items[10]+answ_items[12])

                elif (answ_items[0] == '21'):
                    l_ch3.append(answ_items[2]+answ_items[4])
                    l_ch4.append(answ_items[6]+answ_items[8])
                    l_ch5.append(answ_items[10]+answ_items[12])
                
                elif (answ_items[0] == '41'):
                    l_ch6.append(answ_items[2]+answ_items[4])
                    l_ch7.append(answ_items[6]+answ_items[8])
                    l_ch8.append(answ_items[10]+answ_items[12])

                elif (answ_items[0] == '61'):
                    l_ch9.append(answ_items[2]+answ_items[4])
                    l_ch10.append(answ_items[6]+answ_items[8])
                    l_ch11.append(answ_items[10]+answ_items[12])
            
            #print(l_ch9)

    #add all channel to the list of list to return
    l_allch.extend([l_ch0,l_ch1,l_ch2,l_ch3, l_ch4, l_ch5, l_ch6, l_ch7, l_ch8, l_ch9, l_ch10, l_ch11])

    #return the list of list
    return(l_allch)    



In [342]:
my_allch_list = getADCperchannel(d_list)

In [349]:
#check via printout
print('ch0:', my_allch_list[0], '\t ch1: ', my_allch_list[1], '\t ch2: ', my_allch_list[2])

ch0: ['E6AA', 'E6CC', 'E6A4', 'E683', 'E6CB', 'E6D0', 'E6A4', 'E6B9', 'E6A7', 'E6B0', 'E6C7', 'E6C0', 'E6B9', 'E6D1', 'E6CB', 'E6B3', 'E69A', 'E6AA', 'E6C4', 'E6AC'] 	 ch1:  ['E6BB', 'E6BB', 'E6BC', 'E6BB', 'E6BB', 'E6BD', 'E6BB', 'E6B9', 'E6BA', 'E6BA', 'E6BD', 'E6BB', 'E6BA', 'E6BB', 'E6BA', 'E6B9', 'E6BB', 'E6BC', 'E6BC', 'E6BA'] 	 ch2:  ['E6B4', 'E6B4', 'E6B4', 'E6B3', 'E6B6', 'E6B2', 'E6B3', 'E6B5', 'E6B3', 'E6B3', 'E6B5', 'E6B2', 'E6B6', 'E6B3', 'E6B3', 'E6B3', 'E6B7', 'E6B4', 'E6B4', 'E6B3']


In [416]:
# define a function which read a list of list with all ADC values per channel and do :
# the conversion exadecimal to decimal
# calculate the average
import statistics

#function to convert hex to decimal
def todec(n):
    return int(n, 16) 

#function to convert back to hex (after rounding to get to the closest integer)
def tohex(n):
    return hex(round(n))


#function to get the list of list of all values per channel. Per each channel do: 
# - convert to dec compute the average
# - compute the average
# - store the value in a list (this will contain one value per channel )
# - convert the values in the list to hex 
# - return the list containing all the averages per channel --> these are the values to be used for the calibration campaign
#   

def Getch_average(chlist):
    chlist = chlist.copy()
    print(len(chlist))
    
    ch_avg_dec = []
    ch_avg_hex = []

    for ch in chlist:

        # convert any elements of the channel list to decimal
        singlech_dec = list(map(todec, ch))
        print(singlech_dec)
        
        #compute the average
        ch_avg = statistics.mean(singlech_dec)
        ch_avg_dec.append(ch_avg)

        #print(list(singlech_dec))
        #print(singlech_avg)

    print(ch_avg_dec)
    #convert back to hex and save to a list which will be our final output
    ch_avg_hex = list(map(tohex, ch_avg_dec))
    print(ch_avg_hex)
    return ch_avg_hex

In [418]:
mych_average_biasV = Getch_average(my_allch_list)

12
[59050, 59084, 59044, 59011, 59083, 59088, 59044, 59065, 59047, 59056, 59079, 59072, 59065, 59089, 59083, 59059, 59034, 59050, 59076, 59052]
[59067, 59067, 59068, 59067, 59067, 59069, 59067, 59065, 59066, 59066, 59069, 59067, 59066, 59067, 59066, 59065, 59067, 59068, 59068, 59066]
[59060, 59060, 59060, 59059, 59062, 59058, 59059, 59061, 59059, 59059, 59061, 59058, 59062, 59059, 59059, 59059, 59063, 59060, 59060, 59059]
[59054, 59054, 59055, 59053, 59053, 59055, 59053, 59054, 59051, 59053, 59052, 59053, 59052, 59053, 59054, 59054, 59053, 59052, 59055, 59054]
[59074, 59074, 59074, 59073, 59075, 59074, 59074, 59075, 59073, 59073, 59074, 59074, 59075, 59075, 59072, 59073, 59074, 59073, 59074, 59074]
[59065, 59064, 59067, 59063, 59066, 59065, 59067, 59066, 59064, 59066, 59066, 59064, 59066, 59066, 59066, 59065, 59067, 59067, 59065, 59065]
[59055, 59053, 59053, 59054, 59055, 59054, 59054, 59054, 59054, 59056, 59052, 59054, 59053, 59056, 59056, 59054, 59055, 59054, 59053, 59055]
[59063, 59

In [422]:
def printhex(myhex):
    print(myhex[2:])

print(mych_average_biasV)
hexprint = list(map(printhex, mych_average_biasV))

['0xe6b6', '0xe6bb', '0xe6b4', '0xe6ad', '0xe6c2', '0xe6ba', '0xe6ae', '0xe6b7', '0xe6fa', '0xe6cf', '0xe6d9', '0xe6e0']
e6b6
e6bb
e6b4
e6ad
e6c2
e6ba
e6ae
e6b7
e6fa
e6cf
e6d9
e6e0
